In [2]:
#default_exp boxes

In [3]:
#export

import os, sys
import torch
import numpy as np

from models import build_DABDETR, build_dab_deformable_detr
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops

from PIL import Image
import datasets.transforms as T
import requests, io
import requests
from dab.paths import *

from dab.segment import *
import requests
from PIL import Image
from io import BytesIO


vslzr = COCOVisualizer()

def dtd(d,device):
    return {k: v.to(device=device, non_blocking=True) if hasattr(v, 'to') 
               else dtd(v,device) if hasattr(v, 'items') 
               else [dtd(i,device) for i in v] if isinstance(v,list)
               else v for k, v in d.items()}
model_config_path     = "model_zoo/DAB_DETR/R50_v2/config.json" # change the path of the model config file
model_checkpoint_path = "model_zoo/DAB_DETR/R50_v2/checkpoint.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.

device='cpu'
args = SLConfig.fromfile(model_config_path) 
model, criterion, postprocessors = build_dab_deformable_detr(args)
checkpoint = torch.load(model_checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model'])
model = model.to('cuda')
def dtd(d,device):
    return {k: v.to(device=device, non_blocking=True) if hasattr(v, 'to') 
               else dtd(v,device) if hasattr(v, 'items') 
               else [dtd(i,device) for i in v] if isinstance(v,list)
               else v for k, v in d.items()}

/home/dima/DAB-DETR/STEGO/src/train_segmentation.py:386: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="train_config.yml")


Since no pretrained weights have been provided, we load the reference pretrained DINO weights.


/home/dima/anaconda3/envs/dab/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (UnsupervisedMetrics). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [4]:
from dab.tools import get_image_from_url

In [5]:
#export
def get_boxes(image,thershold=35):

    image_init = image



    # image
    # transform images
    transform = T.Compose([
        T.RandomResize([800], max_size=1333),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image, _ = transform(image_init, None)
    # predict images
    output = model(image[None].to('cuda'))
    output = dtd(output,'cpu')
    output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]))[0]
    # visualize outputs

    scores = output['scores']
    labels = output['labels']
    boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
    select_mask = scores > thershold

    # box_label = [id2name[int(item)] for item in labels[select_mask]]
    pred_dict = {
        'boxes': boxes[select_mask],
        'size': torch.Tensor([image.shape[1], image.shape[2]]),
        # 'box_label': box_label
    }
    # vslzr.visualize(image, pred_dict, savedir=None)
    W,H = image_init.size

    boxes = []

    for box in pred_dict['boxes']:
        unnormbbox = box * torch.Tensor([W, H, W, H])
        unnormbbox[:2] -= unnormbbox[2:] / 2
        [bbox_x, bbox_y, bbox_w, bbox_h] = unnormbbox.tolist()

        boxes.append((bbox_x,bbox_y,bbox_x+bbox_w,bbox_y+bbox_h))


    return boxes

    #

In [6]:

def save_return_boxes(url,thershold=0.35):
    response = requests.get(url)
    image_bytes = io.BytesIO(response.content)
    image_init = Image.open(image_bytes).convert("RGB")
    boxes = get_boxes(image_init,thershold)
    photo_id = url.split('/')[-1]
    for id, box in enumerate(boxes): image_init.crop(box).save(fragment_reference_images_path/f'{photo_id}_{id}.jpg')
    return boxes

In [7]:


def save_return_boxes(url,thershold=0.35):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = get_food_segment(img)
    boxes = get_boxes(img,thershold=thershold)
    photo_id = url.split('/')[-1]
    for id, box in enumerate(boxes): img.crop(box).save(fragment_reference_images_path/f'{photo_id}_{id}.jpg')
    return boxes

In [25]:
url = 'https://image.shutterstock.com/z/stock-photo-club-sandwich-and-french-fries-on-a-light-wooden-board-next-to-the-potatoes-is-a-cup-of-ketchup-1147730354.jpg'

In [26]:
image_init = get_image_from_url(url)

In [27]:
boxes = get_boxes(image_init,thershold=0.00035);

/home/dima/DAB-DETR/models/dab_deformable_detr/position_encoding.py:53: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)
/home/dima/DAB-DETR/models/dab_deformable_detr/deformable_transformer.py:489: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = 10000 ** (2 * (dim

In [28]:
img_array = np.array(image_init)

In [29]:
img_array.shape

(1098, 1500, 3)

In [30]:
boxes = [[int(bb) for bb in b] for b in boxes]
arrays = []
for b in boxes:
    arrays.append()
img_array[~img_array[b[1]:b[3],b[0]:b[2],:]] = 0

In [23]:
img_array[~img_array[444:760,280:577,:]] = 0

In [20]:
image_init.size

(1500, 1098)

In [19]:
img_array.shape

(1098, 1500, 3)

In [22]:
image_init.crop?

Signature: image_init.crop(box=None)
Docstring:
Returns a rectangular region from this image. The box is a
4-tuple defining the left, upper, right, and lower pixel
coordinate. See :ref:`coordinate-system`.

Note: Prior to Pillow 3.4.0, this was a lazy operation.

:param box: The crop rectangle, as a (left, upper, right, lower)-tuple.
:rtype: :py:class:`~PIL.Image.Image`
:returns: An :py:class:`~PIL.Image.Image` object.
File:      ~/anaconda3/envs/dab/lib/python3.9/site-packages/PIL/Image.py
Type:      method


In [70]:
boxes

[(280.15777587890625, 444.86474609375, 577.4803161621094, 760.3456726074219),
 (566.224365234375, 490.0134582519531, 867.5025634765625, 811.2196960449219),
 (838.279052734375, 318.07244873046875, 1145.2294311523438, 614.3818359375),
 (828.1907348632812, 566.5265502929688, 1100.6534729003906, 867.0364379882812),
 (295.9640808105469,
  211.29629516601562,
  511.96702575683594,
  426.9286804199219),
 (296.2718811035156, 211.83364868164062, 511.3861999511719, 426.9445037841797),
 (829.9771118164062, 565.14501953125, 1100.8348083496094, 869.0626831054688),
 (303.3920593261719, 641.294189453125, 379.73597717285156, 696.5904769897461),
 (295.9640808105469,
  211.29629516601562,
  511.96702575683594,
  426.9286804199219),
 (926.1633911132812, 565.0516357421875, 1101.6889343261719, 848.4552917480469),
 (927.90625, 563.393798828125, 1097.5391540527344, 813.7358856201172),
 (323.7318420410156, 642.7954711914062, 379.8770942687988, 690.7076301574707),
 (721.9324951171875, 534.2677001953125, 871.69

In [5]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted boxes.ipynb.
Converted food_segment.ipynb.
Converted tests.ipynb.
